# Tokenization

Este cuaderno muestra, paso a paso, cómo **tokenizar texto con Hugging Face Transformers** usando `AutoTokenizer` (modelo `google-bert/bert-base-cased`) sobre un fragmento de *Don Quixote* tomado de Wikipedia. Tras cargar el tokenizador, ajustamos parámetros clave para el **preprocesado**: `max_length` (límite de tokens del modelo), `truncation` (recorta entradas largas) y `padding` (rellena entradas cortas), incluyendo la práctica de asignar `pad_token = eos_token` cuando el tokenizador no define `PAD` (típico en modelos *decoder-only*). A continuación generamos tensores (`return_tensors="pt"`) y comparamos el **número de palabras** del texto original con la **longitud en tokens** producida, ilustrando que los tokens no tienen por qué coincidir con palabras. Finalmente, inspeccionamos estructuras adicionales que devuelve la tokenización: `token_type_ids`, que identifica el **segmento/rol** de cada token (útil en tareas con pares de textos), y `attention_mask`, que indica qué posiciones deben **recibir atención** (1) y cuáles se **ignoran** (0), normalmente el *padding*.


In [ ]:
# Para este notebook es necesario tener instalada la librería transformer
from transformers import AutoTokenizer

In [47]:
# El texto que vamos a tokenizar es el siguiente, extraido de: https://en.wikipedia.org/wiki/Don_Quixote
text= """
Cervantes, in a metafictional narrative, writes that the first few chapters were taken from "the archives of La Mancha", and the rest were translated from an Arabic text by the Moorish historian Cide Hamete Benengeli.

Alonso Quixano is an hidalgo nearing 50 years of age who lives in a deliberately unspecified region of La Mancha with his niece and housekeeper. While he lives a frugal life, he is full of fantasies about chivalry stemming from his obsession with chivalric romance books. Eventually, his obsession becomes madness when he decides to become a knight errant, donning an old suit of armor. He renames himself "Don Quixote", names his old workhorse "Rocinante", and designates Aldonza Lorenzo (a slaughterhouse worker with a famed hand for salting pork) his lady love, renaming her Dulcinea del Toboso.

As he travels in search of adventure, he arrives at an inn that he believes to be a castle, calls the prostitutes he meets there "ladies", and demands that the innkeeper, whom he takes to be the lord of the castle, dub him a knight. The innkeeper agrees. Quixote starts the night holding vigil at the inn's horse trough, which Quixote imagines to be a chapel. He then becomes involved in a fight with muleteers who try to remove his armor from the horse trough to water their mules. In a pretend ceremony, the innkeeper dubs him a knight to be rid of him and sends him on his way.

Quixote next encounters a servant named Andres who is tied to a tree and being beaten by his master over disputed wages. Quixote orders the master to stop the beating, untie Andres and swear to treat his servant fairly. However, the beating is resumed, and redoubled, as soon as Quixote leaves.

Quixote then chances upon traders from Toledo. He demands that they agree that Dulcinea del Toboso is the most beautiful woman in the world. One of them demands to see her picture so that he can decide for himself. Enraged, Quixote charges at them but his horse stumbles, causing him to fall. One of the traders beats up Quixote, who is left at the side of the road until a neighboring peasant brings him back home.

While Quixote lies unconscious in his bed, his niece, the housekeeper, the parish curate, and the local barber burn most of his chivalric and other books, seeing them as the root of his madness. They seal up the library room, later telling Quixote that it was done by a wizard. """
print(text)


Cervantes, in a metafictional narrative, writes that the first few chapters were taken from "the archives of La Mancha", and the rest were translated from an Arabic text by the Moorish historian Cide Hamete Benengeli.

Alonso Quixano is an hidalgo nearing 50 years of age who lives in a deliberately unspecified region of La Mancha with his niece and housekeeper. While he lives a frugal life, he is full of fantasies about chivalry stemming from his obsession with chivalric romance books. Eventually, his obsession becomes madness when he decides to become a knight errant, donning an old suit of armor. He renames himself "Don Quixote", names his old workhorse "Rocinante", and designates Aldonza Lorenzo (a slaughterhouse worker with a famed hand for salting pork) his lady love, renaming her Dulcinea del Toboso.

As he travels in search of adventure, he arrives at an inn that he believes to be a castle, calls the prostitutes he meets there "ladies", and demands that the innkeeper, whom he t

In [ ]:
# Atributos de la tokenización:

## Podemos cargar el tokenizado de un modelo concreto de la siguiente forma
pretrained_model= "google-bert/bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)


## El tamaño máximo de la lista de tokens a generar se puede ajustar, aunque lo común es usar el tamaño máximo del modelo.
## De esta forma, al realizar tareas de PLN con el texto tokenizado, el modelo trabajará sobre un mayor tamaño de texto
max_length= tokenizer.model_max_length 

## Habilitar la truncación implica que cuando el tokenizador genera más de max_length tokens para un texto,
## este devuelve solo los primeros max_length tokens. 
truncation = True # or False 

## Habilitar el padding implica que cuando un tokenizador genera menos de max_length tokens para un texto,
## este añade tokens vacíos hasta completar los max_length tokens.
padding = True

## En general truncation y padding son estrategias para mantener un número de tokens homogeneo independientemente de los textos.
## De esta forma los textos largos son acortados y los cortos son extendidos. Permitiendo realizar operaciones y transformaciones.


## Hay modelos, como los decoder-only que no tienen un token para realizar el padding (como GPT2). 
## En caso de que se quiera utilizar el tokenizador de estos modelos, la práctica común es utilizar su token de final de sentencia
## para realizar el padding. De esta forma se consigue el mismo resultado.
if tokenizer.pad_token is None: 
    tokenizer.pad_token= tokenizer.eos_token



In [ ]:
# Una vez instanciado nuestro tokenizador podemos utilizarlo de la siguiente forma:
tokenized_text= tokenizer(  text,
                            max_length= max_length,
                            padding=    padding,
                            truncation= truncation,
                            return_tensors= "pt"  # De esta forma lo mantenemos como lista por flexibilidad
                        ).to("cpu") # Se podría hacer en GPU, pero como no es un proceso pesado se puede hacer en CPU.
print(tokenized_text)

In [55]:
# Como podemos ver, el resultado del modelo son una serie de tokens 'input_ids' con los que el modelo bert-base-cased puede trabajar.
# Cada modelo tiene su tokenizador, por lo que los tokens de uno no son compatibles con otro.
# Los tokens no tienen porque representar cada palabra, por eso puede que el tamaño del texto tokenizado sea diferente del número de palabras.

print(f"Cantidad de palabras del texto original: {len(text.split(' '))} \nCantidad de tokens generados por el modelo:{tokenized_text['input_ids'].shape[1]}")


Cantidad de palabras del texto original: 415 
Cantidad de tokens generados por el modelo:512


In [56]:
# Sin embargo, el tokenizador nos puede devolver más cosas que los tokens.

# Por ejemplo tenemos el token_type_ids, que es una lista que define a que segmento pertenece cada token. 
# Esto se utiliza en varias tareas como QA para diferenciar los tokens de la pregunta de la respuesta, o NLI para diferencia hipótesis de premisa.
print(tokenized_text["token_type_ids"])


# Además de contar con la lista que especifíca el segmento al que pertenece el token, también contamos con otra lista que especifíca con que tokens se debe calcular la atención.
# Esta lista, presente en el valor attention_mask, marca como 1 los valores a los que hay que atender durante el cálculo de atención y con 0 los que no (generalmente los tokens de padding).
print(tokenized_text["attention_mask"])

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0